# Introduction
In this assignment, I will be manipulating and examining the XYZ company Data for Marketing and CRM. The data was downloaded from the SSCC as csv files. The files will be titled: Customer, Item, and mail.

# Part 1
The following packages were imported after being installed in order to complete parts 1-5.

In [ ]:
import pandas as pd
import xlrd
import sqlalchemy

The files were imported into padas data frames

In [ ]:
cust_df=pd.read_csv('jpf6752_customer.csv')
type(cust_df)
item_df=pd.read_csv('jpf6752_item.csv')
type(item_df)
mail_df=pd.read_csv('jpf6752_mail.csv')
type(cust_df)

Print the column names and the first four records of the customers, items, and mail data frames

In [ ]:
#Customers
cust_df.shape
cust_df.columns
cust_df.head(4)
#Items
item_df.shape
item_df.columns
item_df.head(4)
#Mail
mail_df.shape
mail_df.columns
mail_df.head(4)

# Part 2
Each of the data frames above were sorted to include active buyers only. They were then imported into a SQLite dattabase names xyz.db.

In [ ]:
cust_df.insert(9,'active_buyer', np.where(cust_df.buyer_status=='ACTIVE',1,2))
cust_df.head(10)

#Pre-sorted DataFrame for active customers, see part 5
cust_df2 = cust_df.loc[cust_df['buyer_status'] == 'ACTIVE']
cust_df2.head(10)

#Import swlaclhemy in order to do the rest of part 2
import sqlalchemy #installed package SQLAlchemy 1.1.6-1
from sqlalchemy import create_engine

Create the engine and check to see what table names are already in the engine. In this case, there are no table names written.

In [ ]:
engine=create_engine('sqlite:///xyz.db')
conn=engine.connect()

engine.table_names()

Write the customers data frame containing only active buyers to the database.

In [ ]:
cust_df.to_sql('Customers',conn,index=False)

count = pd.read_sql_query("SELECT COUNT(*) FROM Customers",conn)
print(count)

Write the Items table and Mail data frames to the xyz database.

In [ ]:
item_df.to_sql('Items',conn,index=False)
mail_df.to_sql('Mail',conn,index=False)

Double checking the engine table names to ensure that they were written correctly.

In [ ]:
engine.table_names()

# Part 3
In part three, a new table called "custsum" was created to model a couple of effects that were placed on the data frame. Parts A-G were combined for ease of writing the code.

Reading the customer data frame and then sorting it to include active buyers only. 

In [ ]:
catsum2 = cust_df
catsum2.head(20)
catsum3 = catsum2[catsum2.buyer_status == 'ACTIVE']
catsum3.head()

Heavy buyers were classified as being in the top 90th percentile or more in total purchases ranging from 2009 year to date.

In [ ]:
catsum4 = catsum3.groupby('acctno')['ytd_sales_2009'].sum().reset_index()
print catsum4
catsum4['heavy_buyer'] = np.where(catsum4['ytd_sales_2009'] >= (catsum4['ytd_sales_2009'].quantile(0.9)), 'Y','N')
catsum4.head(20)

In part B, We were to compile whether or not the customer had used specific credit cards such as AMEX, Discover, Visa and Mastercard. If they had used the credit card when making a purchase, their column value was marked with a Y. If they had not, it was marked with an N. A new cc_info dataframe was created to track this information. Parts C, D, E, and F were also included. The genders of Adult1_g and Adult2_g were included, along with the median income (med_inc), the zip code (zip) and zip+4 code of each customer(zip4). The account number (acctno) was included as well so that the tables could be related to each other.

In [ ]:
cc_info = cust_df2[['acctno','amex_reg','amex_prem','disc_reg','disc_prem','visa_reg','visa_prem','mc_reg','mc_prem','med_inc','adult1_g','adult2_g','zip','zip4']]
cc_info.head()

custsum = pd.merge(catsum4, cc_info, on='acctno')
custsum.head()

custsum['amex_reg'] = np.where(custsum['amex_reg'] == 'Y', 'Y','N')
custsum['amex_prem'] = np.where(custsum['amex_prem'] == 'Y', 'Y','N')
custsum['disc_reg'] = np.where(custsum['disc_reg'] == 'Y', 'Y','N')
custsum['visa_reg'] = np.where(custsum['visa_reg'] == 'Y', 'Y','N')
custsum['visa_prem'] = np.where(custsum['visa_prem'] == 'Y', 'Y','N')
custsum['mc_reg'] = np.where(custsum['mc_reg'] == 'Y', 'Y','N')
custsum['mc_prem'] = np.where(custsum['mc_prem'] == 'Y', 'Y','N')
print custsum

In part G, we had to write the table to the database and ensure that it was written correctly

In [ ]:
custsum.to_sql('Customer_Summary',conn,index=False)
engine.table_names()

# Part 4
In Part 4, we were to create a DataFrame that would be used for target marketing and write out to a headered csv file. For this particular portion, two different dataframes were created by scratch and then merged together to ensure that the information was recorded correctly.

The first to be merged data frame was the unique account numbers with the sum of each product category.

In [ ]:
a_cust = pd.DataFrame(cust_df.groupby('acctno').buyer_status.unique()).reset_index()
a_cust.head(20)

item2_df = item_df
item2_df.head()

c_item = pd.DataFrame(item2_df.groupby('acctno').deptdescr.value_counts())
c_item.head()

cd_item = pd.DataFrame(c_item.unstack(fill_value='N')).reset_index('acctno')
cd_item.head()

The second database to merge with the total customer total was created

In [ ]:
b_item = pd.DataFrame(item_df.groupby('acctno').totamt.sum()).reset_index()
b_item.head()

The two merge tables along with the master merged table was performed below:

In [ ]:
#First Merge#
d_item = pd.merge(cd_item, a_cust, left_on='acctno',right_on='acctno')
d_item.head()
#Master merge
f_item = pd.merge(d_item, b_item, left_on='acctno',right_on='acctno')

The dataframe was then sorted by "Active" buyers

In [ ]:
cust_item = f_item[(f_item.buyer_status=='ACTIVE')]

The final data frame was created and then written to a csv file and then the xyz database

In [ ]:
#Writing to csv#
cust_item.to_csv("Customer_Item_Detail.csv",index=False)

#Writing to xyz database
cust_item.to_sql('Customer_Item_Detail',conn,index=False)
engine.table_names()

# Part 5
Part 5 was split into two separate parts. In the first part we were to find for each gender code for Adult1_g to calculate and report the number of adults with that gender code.

Part 5 was set up by creating a dataframe that would suit our needs. Pre-sorted for active buyers, the second customer data frame was used from part 2. The item dataframe was merged with it to get the necessary information.

In [ ]:
freq_p = cust_df2.merge(item_df)
freq_p.head()

freq_p2 = freq_p[['acctno','adult1_g','totamt','qty','deptdescr']]
freq_p2.head()

The number of adults with specific gender codes was created for Part 1 of Part 5 below:

In [ ]:
check2 = freq_p2.groupby('adult1_g').adult1_g.count()
check2

In Part 2A of Part 5, we were to use the active customer's data to find the top 6 most ordered purchased product categories.

In [ ]:
check3 = freq_p2.groupby('deptdescr').deptdescr.value_counts()
check3

Then, for each of the purchased product categories, the total spent, and the total numnber of products purchased was calculated.

In [ ]:
freq_p2.groupby(by=['adult1_g','deptdescr'])['qty','totamt'].sum()

For Part 2B of part 5, we were to calculate the 6 most common product ordered by gender.

In [ ]:
check = freq_p2.groupby('adult1_g').deptdescr.value_counts()
check
check.groupby(level='adult1_g',group_keys=False).nlargest(6)